In [7]:
import sqlite3
import re

def extract_name(title):
    pattern = r'『([^』]+)』'
    match = re.search(pattern, title)
    if match:
        return match.group(1)
    else:
        return None  # 名前が見つからない場合

In [8]:


def update_database_with_authors(db_path):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    # 全てのタイトルとIDを取得
    cursor.execute("SELECT id, title FROM quotes")
    quotes = cursor.fetchall()

    # 各タイトルから名前を抽出し、データベースを更新
    for quote_id, title in quotes:
        author_name = extract_name(title)
        if author_name:
            cursor.execute("UPDATE quotes SET author = ? WHERE id = ?", (author_name, quote_id))

    # 変更をコミット
    conn.commit()
    conn.close()

# データベースのパスを指定
db_path = 'quotes_20240417_135122_加工用.db'
update_database_with_authors(db_path)


In [1]:

import sqlite3

def check_table_schema(db_path):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    cursor.execute("PRAGMA table_info(quotes)")
    columns = cursor.fetchall()
    conn.close()
    return columns

# データベースのパスを指定
db_path = 'quotes_20240417_135122_加工用.db'
columns = check_table_schema(db_path)
for col in columns:
    print(col)


In [6]:
import sqlite3

def add_id_column(db_path):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()



    # 新しいテーブル構造でテーブルを作成（URLカラムを含む）
    cursor.execute("""
        CREATE TABLE quotes (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            title TEXT,
            quote TEXT,
            author TEXT,
            url TEXT  -- URLカラムを追加
        );
    """)

    # バックアップしたテーブルから新しいテーブルへデータを移行
    cursor.execute("""
        INSERT INTO quotes (title, quote, author, url)
        SELECT title, quote, author, url FROM quotes_backup;
    """)

    # バックアップした古いテーブルを削除（オプション）
    cursor.execute("DROP TABLE quotes_backup;")

    # 変更をコミット
    conn.commit()
    conn.close()

# データベースのパスを指定
db_path = 'quotes_20240417_135122_加工用.db'
add_id_column(db_path)


In [9]:
import sqlite3

def remove_null_rows(db_path, column_name):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    # NULL値を含む行を削除
    cursor.execute(f"DELETE FROM quotes WHERE {column_name} IS NULL;")
    conn.commit()
    conn.close()
    print(f"Removed rows with NULL in {column_name}")

# データベースのパスを指定
db_path = 'quotes_20240417_135122_加工用.db'
# NULL値を削除したいカラム名
column_name = 'author'
remove_null_rows(db_path, column_name)


Removed rows with NULL in author
